In [31]:
import math
import numpy as np
import torch
import torch.nn as nn   
from functorch import jacrev, vmap
import torch.nn.functional as F

In [35]:
body_index = torch.as_tensor(
    [1,2,0] , dtype = torch.long
).squeeze()

one_hot = F.one_hot(body_index, num_classes=4)
f_x = torch.randn(3,1)
f_y = torch.randn(3,1)
force = torch.zeros(3, 4 ,2)

In [39]:
print(one_hot.shape)
force[torch.arange(3), body_index] = torch.cat([f_x, f_y], dim=1)

torch.Size([3, 4])


In [43]:
force

tensor([[[ 0.0000,  0.0000],
         [ 1.7383,  1.1086],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000]],

        [[ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 1.2363, -1.1247],
         [ 0.0000,  0.0000]],

        [[-0.5505,  2.0544],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000]]])